# SQL Subqueries - Lab

## Introduction

Now that you've seen how subqueries work, it's time to get some practice writing them! Not all of the queries will require subqueries, but all will be a bit more complex and require some thought and review about aggregates, grouping, ordering, filtering, joins and subqueries. Good luck!  

## Objectives

You will be able to:

* Write subqueries to decompose complex queries

## CRM Database Schema

Once again, here's the schema for the CRM database you'll continue to practice with.

<img src="images/Database-Schema.png" width="600">

## Connect to the Database

As usual, start by importing the necessary packages and connecting to the database **data.sqlite**.

In [1]:
# Your code here; import the necessary packages
import pandas as pd
import sqlite3
conn = sqlite3.connect('data.sqlite')
cur = conn.cursor()

## Write an Equivalent Query using a Subquery

```SQL
SELECT customerNumber,
       contactLastName,
       contactFirstName
       FROM customers
       JOIN orders 
       USING(customerNumber)
       WHERE orderDate = '2003-01-31';
```

In [3]:
# Your code here; use a subquery. No join is necessary 
cur.execute("""
SELECT customerNumber, contactLastName, contactFirstName
FROM customers
WHERE customerNumber IN 
(SELECT customerNumber
FROM orders
WHERE orderDate = '2003-01-31');
""")
df = pd.DataFrame(cur.fetchall())
df.columns = [x[0] for x in cur.description]
df

,customerNumber,contactLastName,contactFirstName
0,141,Freyre,Diego


## Select the Total Number of Orders for Each Product Name

Sort the results by the total number of items sold for that product.

In [22]:
# Your code here
cur.execute("""
SELECT products.productName, 
COUNT(orderdetails.orderNumber) AS total_num_orders,
SUM(orderdetails.quantityOrdered) AS total_num_units
FROM products
JOIN orderdetails
USING(productCode)
GROUP BY products.productName
ORDER BY total_num_units DESC;
""")
df2 = pd.DataFrame(cur.fetchall())
df2.columns = [x[0] for x in cur.description]
df2.head(10)

,productName,total_num_orders,total_num_units
0,1992 Ferrari 360 Spider red,53,1808
1,1937 Lincoln Berline,28,1111
2,American Airlines: MD-11S,28,1085
3,1941 Chevrolet Special Deluxe Cabriolet,28,1076
4,1930 Buick Marquette Phaeton,28,1074
5,1940s Ford truck,28,1061
6,1969 Harley Davidson Ultimate Chopper,28,1057
7,1957 Chevy Pickup,28,1056
8,1964 Mercedes Tour Bus,28,1053
9,1956 Porsche 356A Coupe,27,1052


## Select the Product Name and the  Total Number of People Who Have Ordered Each Product

Sort the results in descending order.

### A quick note on the SQL  `SELECT DISTINCT` statement:

The `SELECT DISTINCT` statement is used to return only distinct values in the specified column. In other words, it removes the duplicate values in the column from the result set.

Inside a table, a column often contains many duplicate values; and sometimes you only want to list the unique values. If you apply the `DISTINCT` clause to a column that has `NULL`, the `DISTINCT` clause will keep only one NULL and eliminates the other. In other words, the DISTINCT clause treats all `NULL` “values” as the same value.

In [27]:
# Your code here:
# Hint: because one of the tables we'll be joining has duplicate customer numbers, you should use DISTINCT
cur.execute("""
SELECT products.productName, 
COUNT(DISTINCT customers.customerNumber) AS total_num_purchasers
FROM customers
JOIN orders
USING(customerNumber)
JOIN orderdetails
USING(orderNumber)
JOIN products
USING(productCode)
GROUP BY products.productName
ORDER BY total_num_purchasers DESC;
""")
df4 = pd.DataFrame(cur.fetchall())
df4.columns = [x[0] for x in cur.description]
df4.head(10)

,productName,total_num_purchasers
0,1992 Ferrari 360 Spider red,40
1,1934 Ford V8 Coupe,27
2,1952 Alpine Renault 1300,27
3,1972 Alfa Romeo GTA,27
4,Boeing X-32A JSF,27
5,18th Century Vintage Horse Carriage,26
6,1900s Vintage Bi-Plane,26
7,1913 Ford Model T Speedster,26
8,1937 Horch 930V Limousine,26
9,1940 Ford Delivery Sedan,26


## Select the Employee Number, First Name, Last Name, City (of the office), and Office Code of the Employees Who Sold Products Which Have Been Ordered by Less Then 20 people.

This problem is a bit tougher. To start, think about how you might break the problem up. Be sure that your results only list each employee once.

In [49]:
# Your code here
cur.execute("""
SELECT DISTINCT employeeNumber, officeCode, o.city, firstName, lastName
               FROM employees e
               JOIN offices o
               USING(officeCode)
               JOIN customers c
               ON e.employeeNumber = c.salesRepEmployeeNumber
               JOIN orders
               USING(customerNumber)
               JOIN orderdetails
               USING(orderNumber)
               WHERE productCode IN (SELECT productCode
                                            FROM products
                                            JOIN orderdetails
                                            USING(productCode)
                                            JOIN orders
                                            USING(orderNumber)
                                            GROUP BY productCode
                                            HAVING COUNT(DISTINCT customerNumber) < 20);
""")
df5 = pd.DataFrame(cur.fetchall())
df5.columns = [x[0] for x in cur.description]
df5.head(10)

,employeeNumber,officeCode,city,firstName,lastName
0,1370,4,Paris,Gerard,Hernandez
1,1501,7,London,Larry,Bott
2,1337,4,Paris,Loui,Bondur
3,1166,1,San Francisco,Leslie,Thompson
4,1286,3,NYC,Foon Yue,Tseng
5,1612,6,Sydney,Peter,Marsh
6,1611,6,Sydney,Andy,Fixter
7,1401,4,Paris,Pamela,Castillo
8,1621,5,Tokyo,Mami,Nishi
9,1323,3,NYC,George,Vanauf


## Select the Employee Number, First Name, Last Name, and Number of Customers for Employees Whose Customers Have an Average Credit Limit of Over 15K

In [58]:
# Your code here
cur.execute("""
SELECT employees.employeeNumber, employees.firstName, employees.lastName,
COUNT(customers.customerNumber) AS total_num_customers, 
customers.creditLimit
FROM employees
JOIN customers
ON employees.employeeNumber = customers.salesRepEmployeeNumber
GROUP BY employees.employeeNumber
HAVING AVG(customers.creditLimit) > 15000
ORDER BY customers.creditLimit DESC
""")
df6 = pd.DataFrame(cur.fetchall())
df6.columns = [x[0] for x in cur.description]
df6

,employeeNumber,firstName,lastName,total_num_customers,creditLimit
0,1165,Leslie,Jennings,6,210500
1,1286,Foon Yue,Tseng,7,138500
2,1337,Loui,Bondur,6,123900
3,1611,Andy,Fixter,5,117300
4,1323,George,Vanauf,8,114900
5,1621,Mami,Nishi,5,103800
6,1216,Steve,Patterson,6,100600
7,1612,Peter,Marsh,5,97900
8,1501,Larry,Bott,8,96500
9,1401,Pamela,Castillo,10,83400


## Summary

In this lesson, you got to practice some more complex SQL queries, some of which required subqueries. There's still plenty more SQL to be had though; hope you've been enjoying some of these puzzles!